In [1]:
import numpy as np
import os
import tensorflow as tf
import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
# take data
data_dir='data'

In [4]:
no_of_videos=75
no_of_frames=30

In [6]:
actions=os.listdir(data_dir)
actions

['blank', 'hello', 'how are you', 'thank you']

In [7]:
action_map={action:num for num,action in enumerate(actions)}

In [8]:
action_map

{'blank': 0, 'hello': 1, 'how are you': 2, 'thank you': 3}

In [18]:
x_data=[]
y_data=[]
for action in tqdm.tqdm(actions):
    index=action_map[action]
    for video in range(no_of_videos):
        window=[]
        for frame in range(no_of_frames):
            path=os.path.join(data_dir,action,str(video),f'{frame}.npy')
            res=np.load(path)
            window.append(res)
        x_data.append(window)
        y_data.append(index)
        
    
            

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [04:03<00:00, 60.89s/it]


In [17]:
len(window)

30

In [21]:
len(y_data),4*75

(300, 300)

In [22]:
x=np.array(x_data)

In [26]:
y=keras.utils.to_categorical(np.array(y_data)).astype(int)

In [30]:
y

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]])

In [31]:
# create model

In [35]:
model=keras.Sequential()
model.add(keras.layers.LSTM(128,return_sequences=True,activation='relu',input_shape=(30,1662)))
model.add(keras.layers.LSTM(64,return_sequences=True,activation='relu'))
model.add(keras.layers.LSTM(32,return_sequences=False,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(4,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           916992    
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            49408     
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 4)                 132       
                                                                 
Total params: 983140 (3.75 MB)
Trainable params: 983140 

In [37]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.15,stratify=y)

In [38]:
x_train.shape,y_train.shape

((255, 30, 1662), (255, 4))

In [39]:
history1=model.fit(x_train,y_train,epochs=150,validation_data=(x_test,y_test))

Epoch 1/150
8/8 [==============================] - 13s 393ms/step - loss: 15.1333 - categorical_accuracy: 0.2588 - val_loss: 13.0245 - val_categorical_accuracy: 0.2444
Epoch 2/150
8/8 [==============================] - 2s 248ms/step - loss: 46.6222 - categorical_accuracy: 0.2431 - val_loss: 45.6941 - val_categorical_accuracy: 0.2889
Epoch 3/150
8/8 [==============================] - 2s 223ms/step - loss: 18.6062 - categorical_accuracy: 0.2980 - val_loss: 13.1655 - val_categorical_accuracy: 0.2667
Epoch 4/150
8/8 [==============================] - 2s 227ms/step - loss: 13.3822 - categorical_accuracy: 0.2745 - val_loss: 15.6647 - val_categorical_accuracy: 0.1333
Epoch 5/150
8/8 [==============================] - 2s 224ms/step - loss: 17.9730 - categorical_accuracy: 0.2353 - val_loss: 20.6338 - val_categorical_accuracy: 0.2222
Epoch 6/150
8/8 [==============================] - 2s 230ms/step - loss: 45.7367 - categorical_accuracy: 0.2196 - val_loss: 78.2287 - val_categorical_accuracy: 0.24